In [1]:
from src.training.ModelTrainer import ModelTrainer
from src.datasets.shape_net.shape_net_v3_sdf import ShapeNetV3SDF
from src.utils.util import seed_all
from src.utils.visualizations import visualize_sdf_as_mesh
from tqdm.notebook import tqdm as tqdm
import numpy as np
import torch
%load_ext autoreload
%load_ext tensorboard
%autoreload 2
seed_all(111)
mem = torch.cuda.mem_get_info()
print("Memory Before: %s", mem)
trainer = ModelTrainer(dataset_type=ShapeNetV3SDF,
                       options={"tdm_notebook": True})

dataset = trainer.data_loader_handler.dataset
print("Dataset length: ", len(dataset))
mem = torch.cuda.mem_get_info()
print("Memory After: %s", mem)
model = trainer.model


Memory Before: %s (10120527872, 10385358848)
Using device: cuda:0
{'name': 'globalPVQVAEVGGDisc', 'description': 'plz', 'experiment_id': 'vis', 'extra_notes': 'none', 'logs_dir': 'logs', 'is_train': True, 'device': 'cuda:0', 'batch_size': 16, 'num_workers': 16, 'test_size': 0.1, 'n_epochs': 50000, 'append_loss_every': 250, 'print_every': 500, 'validate_every': 1973, 'save_every': 1973, 'save_every_nepochs': 5, 'start_epoch': 0, 'start_iteration': 0, 'visualize_every': 1000, 'apply_metrics_every': 1973, 'load_ckpt': True, 'ckpt_path': 'logs/globalPVQVAEVGGDisc/LinearDiscExp3G1.0e-2@25kBoth/2024_07_10_19_13_37/checkpoints/epoch-best.ckpt', 'use_scheduler': True, 'apply_metrics_batch_count': 5}


- Creating new directory logs/globalPVQVAEVGGDisc/vis/2024_07_11_20_23_33
- Creating new directory logs/globalPVQVAEVGGDisc/vis/2024_07_11_20_23_33/checkpoints
- Creating new directory logs/globalPVQVAEVGGDisc/vis/2024_07_11_20_23_33/tb
- Creating new directory logs/globalPVQVAEVGGDisc/vis/2024_07_11_20_23_33/visuals
- Creating new directory logs/globalPVQVAEVGGDisc/vis/2024_07_11_20_23_33/modelsummary


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[*] Enc has Attn at i_level, i_block: 3, 0
Decoding of shape (1, 256, 8, 8, 8) = 131072 dimensions.
[*] Dec has Attn at i_level, i_block: 3, 0
VGG loaded from logs/VGG/trainFull/2024_07_01_17_50_17/checkpoints/epoch-36.ckpt
Using device: cuda:0
Initializing model weights with normal initialization
Model loaded from logs/globalPVQVAEVGGDisc/LinearDiscExp3G1.0e-2@25kBoth/2024_07_10_19_13_37/checkpoints/epoch-best.ckpt
Model size: 296.617MB
Total_params: 77.7M
Dataset length:  6579
Memory After: %s (7450853376, 10385358848)


In [ ]:
# from src.training.SDFVisualizer import SDFVisualizer
# from src.metrics.pytorch_3d_chamfer_dist import Pytorch3DChamferDistance
# from src.metrics.signed_iou import SignedIou
# from src.metrics.pytorch_3d_chamfer_dist import Pytorch3DChamferDistance
# from src.metrics.signed_iou import SignedIou
# chm = Pytorch3DChamferDistance()
# iou = SignedIou()
# #path = "./src/testVisuals/vggPvqvaeGlobal"
# path = "./src/testVisuals/gan2"
# data_loader = trainer.data_loader_handler.validation_dataloader
# with torch.no_grad():
#     chamferDistTotal = 0
#     iouTotal = 0
#     i = 0
#     for index, batch in tqdm(enumerate(data_loader), total=len(data_loader)):
#         ShapeNetV3SDF.move_batch_to_device(batch, "cuda:0")
#         sdfs = batch['sdf']
#         pred = model.inference(sdfs)
#         visualizer = SDFVisualizer("cuda:0",path,"best", 0, index)
#         visualizer.visualize(pred)
#         chamferDistTotal += chm.calc_batch(pred,sdfs)
#         iouTotal += iou.calc_batch(pred, sdfs)
#         i+=1
# chamferDistTotal =  chamferDistTotal/(i)
# iouTotal = iouTotal/(i)

In [ ]:
iouTotal

In [ ]:
chamferDistTotal

In [5]:
from src.training.SDFVisualizer import SDFVisualizer
from src.metrics.pytorch_3d_chamfer_dist import Pytorch3DChamferDistance
from src.metrics.signed_iou import SignedIou
from src.metrics.pytorch_3d_chamfer_dist import Pytorch3DChamferDistance
from src.metrics.signed_iou import SignedIou
from einops import rearrange
chm = Pytorch3DChamferDistance()
iou = SignedIou()
#path = "./src/testVisuals/vggPvqvaeGlobal"
path = "./src/testVisuals/gan2"
data_loader = trainer.data_loader_handler.validation_dataloader
with torch.no_grad():
    chamferDistTotal = 0
    iouTotal = 0
    i = 0
    for index, batch in tqdm(enumerate(data_loader), total=len(data_loader)):
        ShapeNetV3SDF.move_batch_to_device(batch, "cuda:0")
        sdfs = batch['z_q_global']
        indices = batch['idx_global']
        #sdfs = np.random.uniform(low=-1,high=1,size=(16,256,8,8,8))
        #sdfs =  np.random.uniform(size=(16,256,8,8,8))
        #sdfs = np.ones((16,1,64,64,64))
        #sdfs = torch.from_numpy(sdfs).float().to("cuda:0")
        #sdfs = torch.randn_like(sdfs)
       # pred = model.inference(sdfs)
        #x = torch.randint(0, 512, (16,8,8,8)).to("cuda:0")
        #import pdb;pdb.set_trace()
        #pred = model.decode_enc_indices(sdfs.flatten(1).to("cuda:0"))
        #import pdb;pdb.set_trace()
        random_indices = torch.randint(0,512, (16,8,8,8) )
        #indices = indices.flatten(1)
#         indices[:,random_indices] = torch.randint_like(random_indices, 0,512)
#         indices = rearrange(indices, 'bs (z1 z2 z3)-> bs z1 z2 z3',z1=8, z2=8,z3=8)
        
        pred = model.decode_from_quant(indices.to("cuda:0"))
        #pred = model.decode(sdfs)
        visualizer = SDFVisualizer("cuda:0",path,"decIndices", 0, index)
        visualizer.visualize(pred)
        #chamferDistTotal += chm.calc_batch(pred,sdfs)
        #iouTotal += iou.calc_batch(pred, sdfs)
        i+=1
#chamferDistTotal =  chamferDistTotal/(i)
#iouTotal = iouTotal/(i)

  0%|          | 0/41 [00:00<?, ?it/s]

./src/testVisuals/gan2/decIndices/epoch_0_iter_0.png saved
./src/testVisuals/gan2/decIndices/epoch_0_iter_1.png saved
./src/testVisuals/gan2/decIndices/epoch_0_iter_2.png saved
./src/testVisuals/gan2/decIndices/epoch_0_iter_3.png saved
./src/testVisuals/gan2/decIndices/epoch_0_iter_4.png saved
./src/testVisuals/gan2/decIndices/epoch_0_iter_5.png saved
./src/testVisuals/gan2/decIndices/epoch_0_iter_6.png saved
./src/testVisuals/gan2/decIndices/epoch_0_iter_7.png saved
./src/testVisuals/gan2/decIndices/epoch_0_iter_8.png saved
./src/testVisuals/gan2/decIndices/epoch_0_iter_9.png saved
./src/testVisuals/gan2/decIndices/epoch_0_iter_10.png saved
./src/testVisuals/gan2/decIndices/epoch_0_iter_11.png saved
./src/testVisuals/gan2/decIndices/epoch_0_iter_12.png saved
./src/testVisuals/gan2/decIndices/epoch_0_iter_13.png saved
./src/testVisuals/gan2/decIndices/epoch_0_iter_14.png saved
./src/testVisuals/gan2/decIndices/epoch_0_iter_15.png saved
./src/testVisuals/gan2/decIndices/epoch_0_iter_16.

In [ ]:
  sdfs = np.random.uniform(low=-1,high=1,size=(16,1,64,64,64))

In [ ]:
sdfs.shape

In [ ]:
torch.randInt

In [ ]:
x = torch.randint(0, 10, (16,512))

In [ ]:
x[0]